### Today's lecture

1. Transactions
2. Views
3. Access Rights in SQL

# Transactions

* A logical unit of work consisting of one or more SQL statements
* Atomic transaction
  * __Fully executed__ or
  * __Rolled back__ as if it never occurred
* Isolation from concurrent transactions
  * Changes made by a transaction are not visible to other concurrently executing transactions until the transaction completes

* Transaction model based on two SQL statements:
  * __COMMIT__
  * __ROLLBACK__
* Transactions begin implicitly
  * Ended by commit work or rollback work
* Default on most databases: each SQL statement commits automatically
* Can turn off auto commit for a session (e.g. using API)

# VIEWS

* Definition
* View Creation and Destruction
* Updating Views
* Types of Views

## Views

* One database often supports multiple applications
  * Slightly different pictures of the world.
* __Views__ help accommodate this variation without storing redundant data.

## Views

* In some cases, it is not desirable for all users to see the entire logical model (that is, all the actual relations stored in the database.)
>__Example:__
```mysql
Employee(ssn, name, department, project, salary)
```
Consider a person who needs to know the name and project of employees in the 'Development' department, but not the salary.  This person should see a relation described, in SQL, by
```mysql
SELECT name, project
FROM   Employee
WHERE  department = 'Development'
```

## Views

* Provide a mechanism to hide certain data from the view of certain users.

* Any relation that is not part of the conceptual model but is visible to a user as a _“virtual relation”_ is called a __view__.

* Not physically stored.

## Levels of Abstraction

<img style="float: right;" src="./figures/levels_of_abstraction.png" width="40%">

* Multiple views
* A single Conceptual (Logic) Schema
* A single Physical Schema


## Levels of Abstraction

### Physical Level

<img style="float: right;" src="./figures/levels_of_abstraction.png" width="40%">

* Lowest level
* <span style="color:red">How the data is physically stored</span>
* It includes
  * Where the data is located
  * File structures
  * Access methods
  * Indexes
* Managed by the __Database Administrator__ 

## Levels of Abstraction

### Conceptual or Logical Level

<img style="float: right;" src="./figures/levels_of_abstraction.png" width="40%">

* Middle level
* <span style="color:red">What data</span> is in the DB
* It consists of the schemas described <br>
  with CREATE TABLE statements
  * Has all the data in the DB
  * Has no information on what a <br>
    user views at external level
* Managed by __Database Designers__

## Levels of Abstraction

### External or View Level 

<img style="float: right;" src="./figures/levels_of_abstraction.png" width="40%">

* Highest Level
* Combination of base tables and views
* Views define how certain <br>
  Users/Groups see data:
  * Full or partial data based on the <br>
    business requirement
  * Users have different views, based <br>
    on their levels of access rights
* Exposed to __Users/Applications__ and <br> __Database Designers__.

## Data Independence

<img style="float: right;" src="./figures/levels_of_abstraction.png" width="40%">

* A database model exhibits <br> data independence if:

> Application programs are protected <br>
  from changes in the conceptual and <br>
  physical schemas.

* Why is this important?
  * _Everything changes._


* Each higher level of the data architecture <br> is immune to changes of the next lower <br> level of the architecture.

## Data Independence Types

<img style="float: right;" src="./figures/levels_of_abstraction.png" width="40%">

* __Physical data independence__ 
  * Can modify the _physical schema_ <br> without causing application <br> programs to be rewritten.


* __Logical data independence__
  * Can modify the _logical schema_ <br> without causing application <br> program to be rewritten.

## View Creation and Destruction

* A view is defined using the create view statement which has the form
```mysql
CREATE VIEW view_name AS
< QUERY >
[WITH CHECK OPTION]
```
where < query > is any legal SQL expression.

* Once a view is defined, the view name can be used to refer to the __virtual relation__ that the view generates.

## View Creation and Destruction

* __CHECK OPTION__
  * Ensures that all UPDATE and INSERTs operations satisfy the condition(s) in the view definition.
  * Otherwise, the UPDATE or INSERT returns an error.
  * Not implemented in SQLite


* __Destruction__
```mysql
DROP VIEW <view_name>
```

## Views for Security

> __Example:__
  ```mysql
  Student(studID, name, address, major, gpa)
  ```
  * This is a view of the Student table without the gpa field.
    ```mysql
    CREATE VIEW SecStudent AS
    SELECT studID, name, address, major
    FROM   student
    ```

## Views for Extensibility

> __Example:__
  * A company's database includes a relation: 
  ```mysql
  Part (PartID, weight,...)
  ```
  * Weight is stored in pounds
  * The Company is purchased by a firm that uses metric weights
  * Databases must be integrated and use Kg.
    * But old applications use pounds.

## Views for Extensibility

> * Solution:
  * Base table with kilograms becomes _MetricPart_ for the integrated company
  ```mysql
  CREATE VIEW MetricPart AS
  SELECT PartID, 2.2046*weight,       -- no other changes
  FROM Part
  ```
  * Old programs still call the table _Part_

## Data Partitioning

* Sometimes the data of a database is partitioned.


* __Horizontal:__ projection on certain attributes
  * Break up our table based on rows
  * Useful when some attributes are bulky or rarely used
  * Distributed databases


* __Vertical:__ selection on certain values (ClientsParis, ClientsLyon)
  * Splitting out extra columns into their own table(s)

## Another Example

> 
* Consider the following relations
  ```mysql
  Person(name, city)
  Purchase(buyer, seller, product, store)
  Product(name, maker, category)
  ```
  ```mysql
  CREATE VIEW  SeattleView  AS
  SELECT buyer, seller, product, store
  FROM   Person, Purchase
  WHERE  Person.city = 'Seattle'
         AND Person.name = Purchase.buyer
  ```
* We have a new 'virtual table':
  ```mysql
  SeattleView(buyer, seller, product, store)
  ```

## Using (Querying) a View
* Transparency for the user
  * Handled as tables in the database
* Simplify the user's queries
* Useful in architectures client-server

<img src="./figures/query_view.png" width="60%">

## Using (Querying) a View

> Query using SeatleView
  ```mysql
  SeattleView(buyer, seller, product, store)
  Product(name, maker, category)
  ```
  ```mysql
  SELECT name, store
  FROM   SeattleView, Product
  WHERE  SeattleView.product = Product.name
         AND Product.category = 'shoes'
  ```

* When you enter a query that mentions a __view__ in the FROM clause, the DBMS expands/rewrites your query to include the view definition.

## View Expansion

* Query using a __view__
```mysql
SELECT name, SeattleView.store
FROM   SeattleView, Product
WHERE  SeattleView.product = Product.name
       AND Product.category = 'shoes'
```

* Expanded query
```mysql
SELECT name, Purchase.store
FROM   Person, Purchase, Product
WHERE  Person.city = 'Seattle'
       AND Person.name = Purchase.buyer
       AND Purchase.product = Product.name
       AND Product.category = 'shoes'
```

## Another Example

* Query using a __view__
  ```mysql
  SELECT  buyer, seller
  FROM    SeattleView
  WHERE   product= 'gizmo'
  ```

* Expanded query
  ```mysql
  SELECT buyer, seller
  FROM   Person, Purchase
  WHERE  Person.city = 'Seattle'
         AND Person.name = Purchase.buyer
         AND Purchase.product= 'gizmo'
  ```

## Views Defined Using Other Views

* One view may be used in the expression defining another view 
* A view relation $v_1$ is said to <span style="color:red"><b>depend directly on</b></span> a view relation $v_2$  if $v_2$ is used in the expression defining $v_1$
* A view relation $v_1$ is said to <span style="color:red"><b>depend on</b></span> view relation $v_2$ if either $v_1$ depends directly on $v_2$ or there is a path of dependencies from $v_1$ to $v_2$ 
* A view relation $v$ is said to be <span style="color:red"><b>recursive</b></span> if it depends on itself.

## Updating Views

* How can we insert a tuple into a table that _doesn't exist_?

> 
```mysql
Employee(ssn, name, department, project, salary)
```
```mysql
CREATE VIEW  Developers AS
   SELECT name, project
   FROM  Employee
   WHERE department = 'Development'
```

## Updating Views

* How can we insert a tuple into a table that _"doesn't exist"_?

> __Example:__
```mysql
Employee(ssn, name, department, project, salary)
Developers(name, project)
```
The following insertion:
```mysql
INSERT INTO  Developers   
VALUES('Joe', 'Optimizer')
```
becomes:
```mysql
INSERT INTO  Employee 
VALUES(NULL, 'Joe', NULL, 'Optimizer', NULL)
```

## Non-Updateable Views

> 
* Consider the relations
  ```mysql
  Person(name, city)
  Purchase(buyer, seller, product, store)
  ```
  and the view
  ```mysql
  CREATE VIEW  SeattleView  AS
  SELECT seller, product, store
  FROM   Person, Purchase
  WHERE  Person.city = 'Seattle'
         AND Person.name = Purchase.buyer
  ```
* How can we add the following tuple to the view?
    ```mysql
    ('Joe',  'Shoe Model 12345',  'Nine West')
    ```

* We need to add 'Joe' to _Person_ first.
  * How?!
  * One time?
  * Multiple times?

## Updating Views

* Most SQL implementations allow updates <span style="color:red"><b>only on simple views</b></span>. 
  * The ```FROM``` clause has only one database relation.
  * The ```SELECT``` clause contains only attribute names of the relation.
    * No expressions, aggregates, or distinct specification.
  * Any attribute not listed in the ```SELECT``` clause can be set to ```NULL```.
  * The query does not have a ```GROUP BY``` or ```HAVING``` clause.

## Updating Views

* SQLite views are read-only and thus you may not be able to execute a DELETE, INSERT or UPDATE statement on a view.
  * A workaround exists
  * Not in the scope of this class
  * This is why WITH CHECK OPTION is not implemented

## Types of Views

* __Virtual views__
  * Used in databases
  * Computed only __on-demand__ – _slow at runtime_
  * Always up to date

## Types of Views

* __Materialized views__
  * A physical table containing all the tuples in the result of the query defining the view
  * Used in Data Warehouses (but recently also in DBMS)
  * Precomputed offline – _fast at runtime_
  * If relations used in the query are updated, the materialized view result becomes out of date
    * Need to maintain the view, by updating the view whenever the underlying relations are updated.

## Data Warehouse

* A relational database designed for query and analysis rather than for transaction processing.
* Usually contains historical data derived from transaction data.
* Separates analysis workload from transaction workload.
* Enables an organization to consolidate data from several sources.

## Advantages/Disadvantages of Views

|     ADVANTAGES     |     DISADVANTAGES     |
|:------------------:|:---------------------:|
|  Data independence |   Update restriction  |
|      Currency      | Structure restriction |
|  Improved security |      Performance      |
| Reduced complexity |                       |
|     Convenience    |                       |
|    Customization   |                       |
|   Data integrity   |                       |

## Summary

* A view is a stored query definition
* Views can be very useful
  * Privacy
  * Easier query writing
  * Extensibility
* Not all views can be updated unambiguously
* Three levels of abstraction in a relational DBMS
  * Yields data independence, logical and physical

# Access Rights in SQL

* The SQL security model
* Granting and revoking privileges

## Discretionary Access Control

* Each user is given appropriate access rights (_privileges_) on specific DB objects
* Explicit grant of rights on objects to individuals.
* Users obtain certain privileges when they create an object
  * Can _pass_ some or all of these privileges to other users __at their discretion__
* Although flexible, can be circumvented by devious unauthorized user tricking an authorized user into revealing sensitive data.

## Terminology

* <span style="color:red"><b>Privacy</b></span> Users should not be able to see and use data they are not supposed to.
  > e.g., A student can't see other students' grades.
* <span style="color:red"><b>Security</b></span> No one should be able to enter the system and / or impact its behavior without being authorized to do so.
  > e.g., Delete or change data without being authorized
* <span style="color:red"><b>Integrity</b></span> Authorized users should not be able to modify things they are not supposed to.
  >e.g., Only instructors can assign grades.
* <span style="color:red"><b>Availability</b></span> Users should be able to see and modify things they are allowed to.
  > e.g. The DB should always be operational

## SQL Security Related Terminology

* <span style="color:red"><b>User</b></span>
  * Not the schema object, just a name for a session of an individual user
  * Identification by Authorization ID (user name)
* <span style="color:red"><b>Role</b></span>
  * Name for a role, to which rights may be assigned
  * May be granted to users / applications
* <span style="color:red"><b>Privileges</b></span> (Rights)
  * System privileges
  * Object (data) privileges: creator has all privileges
* <span style="color:red"><b>Operations</b></span>
  * GRANT < privilege >
  * REVOKE < privilege >

## Roles and Users

* __Roles__ define a set of privileges for a (potentially) large set of __Users__
```mysql
CREATE ROLE sales_people;
-- grant some privileges to sales_people
-- grant sales_people role to users
```
* Much more economic than direct privileges
* Roles may be assigned to roles
* Often assigned to applications instead of individual users

## Privileges

* Right to perform SQL statement type on objects
* Assigned to users or roles (authorization IDs)
* _Creator of object_: all privileges for that object
* _Administrator_: management of system privileges

## Privileges

* The privileges defined by the ISO standard:
  * SELECT - retrieve data from a table
  * INSERT - insert new rows into a table
  * UPDATE - modify rows of data in a table
  * DELETE - delete rows of data from a table
  * REFERENCE - reference columns of a named table in integrity constraints
  * USAGE - use domains, collations, character sets, and translations

## Grant Privileges

* Syntax
```mysql
GRANT <privileges> ON <object>
TO [<users>|<role>]
[WITH GRANT OPTION]
```

* GRANT OPTION: Right to pass privilege on to other users
  * Only owner can execute CREATE, ALTER, and DROP

> __Example:__ Privilege to INSERT particular columns in a table
```mysql
GRANT INSERT
ON <tablename(<attributenames>)>
TO <users>
[WITH GRANT OPTION]
```

* Access matrix : < user > has < right > on < object >

## Examples

> ```mysql
GRANT INSERT, SELECT ON Movie TO Klaus
```
* Klaus can query __'Movie'__ or insert tuples into it.

___
> ```mysql
GRANT DELETE ON Movie TO shop_owner WITH GRANT OPTION
```
* Anna can delete __'Movie'__ tuples, and also authorize others to do so

___
> ```mysql
GRANT UPDATE (price_Day) ON Movie TO movie_staff
```
* Staff can update (only) the price field of __'Movie'__ tuples

___
> ```mysql
GRANT SELECT ON MovieView TO Customers
```
* This does NOT allow the customers to query __'Movie'__ directly!


## Revoke Privileges

* Syntax
```mysql
REVOKE <privileges>
ON <object>
FROM <users>
[RESTRICT | CASCADE]
```
* RESTRICT: only revoke if none of the privileges have been granted by these users.
* CASCADE: revoke from all users that have been granted the privilege by these users.
* Privilege given from different users must be revoked from all users to loose privilege.

## Summary

* Security of DB and their applications is extremely important.
* Roles make privileges with many users manageable.
* Views also play an important role.
* Fine granular access restriction on objects is very important.



<center><img src="./figures/sql-xkcd.png" width="40%"></center>

<sub>https://www.xkcd.com/327/</sub>

---

In [1]:
# Modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()